In [1]:
from auto_elm import AUTO_ELM 
#from os_MFB1 import MFB_ELM 
# Our proposed algorithm
import pandas as pd 
import math
import numpy as np
import tensorflow as tf
#from tqdm import tqdm_notebook as tqdm # Jupyter notebook should use this
import matplotlib.pyplot as plt
import os
import cv2
import scipy.io as scipy_io

def softmax(a):
    c = np.max(a, axis=-1).reshape(-1, 1)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a, axis=-1).reshape(-1, 1)
    return exp_a / sum_exp_a

def sigmoid(x):
    '''input x should be a 1D array!'''
    for i in range(len(x)):
        x[i] = 1/(1+math.e**(-x[i]))
    return x

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train_dataset = np.array(pd.read_csv("UCI dataset/acoustic_scale", header = None,delimiter=' '))
test_dataset = np.array(pd.read_csv("UCI dataset/acoustic_scale.t",header = None, delimiter=' '))
print(test_dataset)

[[2 '1:-0.962576' '2:-0.960422' ... '49:-0.939076' '50:-0.972626' nan]
 [3 '1:-0.735629' '2:-0.113086' ... '49:-0.99538' '50:-0.993334' nan]
 [3 '1:-0.981124' '2:-0.968923' ... '49:-0.996101' '50:-0.995813' nan]
 ...
 [1 '1:-0.996812' '2:-0.990494' ... '49:-0.937852' '50:-0.939699' nan]
 [1 '1:-0.995711' '2:-0.983737' ... '49:-0.999353' '50:-0.998983' nan]
 [3 '1:-0.987073' '2:-0.992241' ... '49:-0.993805' '50:-0.991061' nan]]


In [3]:
label = train_dataset[:,0]
data = train_dataset[:,1:51]

label = label.reshape(label.shape[0], 1)

label1 = np.where(label==1, 0, label)
label1 = np.where(label1==2, 1, label1)
label1 = np.where(label1==3, 2, label1)
print(label1)

data1 = []
for i in data:
    data2 = []
    for j in i:
        [_, value] = j.split(":")
        data2.append(value)
    data1.append(data2)

    
data = np.array(data1)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(data)
data = scaler.transform(data)
print(data)
data = np.concatenate((label1, data), axis = 1)

[[1]
 [2]
 [2]
 ...
 [1]
 [2]
 [2]]


/usr/lib/python3/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype <U12 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


[[1.00685638e-01 3.20959944e-01 2.68426627e-01 ... 4.56000000e-04
  8.16274714e-04 4.60000000e-04]
 [2.71507465e-01 5.02411141e-01 1.87505898e-01 ... 1.25150000e-03
  1.49056440e-03 1.24800000e-03]
 [3.15320505e-02 8.35403759e-03 3.85195956e-02 ... 1.29050000e-03
  1.60916366e-03 1.42550000e-03]
 ...
 [3.06402995e-03 2.79746259e-02 1.80337022e-02 ... 2.16500000e-03
  2.93992530e-03 1.52500000e-03]
 [2.12955918e-01 3.85491235e-01 3.57333981e-01 ... 5.61000000e-04
  6.12484437e-04 6.17000000e-04]
 [3.39454118e-01 4.91604712e-01 4.40571928e-02 ... 9.46000000e-04
  5.35088677e-04 5.36500000e-04]]


In [4]:
data = np.array(data)
print(data.shape)

(78823, 51)


In [8]:
x_train = data[0:60000,1:]
y_train = data[0:60000,0]
x_test = data[60000:,1:]
y_test = data[60000:,0]
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 50)
(18823, 50)
(60000,)
(18823,)


In [9]:
n_input_nodes = 50
n_hidden_nodes = 2  # used to be 6
n_output_nodes = 50

import time
accuracy_sum = 0
time_sum = 0
for PPAP in range(10):
	tf.reset_default_graph()

	"""BP autoencoder"""
	X = tf.placeholder(tf.float32, [None, n_input_nodes])
	Y = tf.placeholder(tf.float32, [None, n_output_nodes])

	encoding_layer = tf.layers.dense(inputs=X,units=n_hidden_nodes,activation=tf.math.sin)
	Y_hat = tf.layers.dense(inputs=encoding_layer,units=n_output_nodes,activation=None)

	loss = tf.losses.mean_squared_error(labels=Y,predictions=Y_hat)

	optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

	sess = tf.Session()
	sess.run(tf.global_variables_initializer())

	t1 = time.time()
	BATCH_SIZE = 128
	for epoch in range(100):
		i = 0
		while i < len(x_train):
			try:
				batch_x = x_train[i:i+BATCH_SIZE]
				batch_y = x_train[i:i+BATCH_SIZE]
			except:
				batch_x = x_train[i:]
				batch_y = x_train[i:]
			i+=BATCH_SIZE
			sess.run(optimizer, feed_dict={X:batch_x, Y:batch_y})
	t2 = time.time()
	time_sum+=(t2-t1)

	"""classification"""
	x_train_encoded = sess.run(encoding_layer, feed_dict={X: x_train})
	x_test_encoded = sess.run(encoding_layer, feed_dict={X: x_test})

	X = tf.placeholder(tf.float32, [None, n_hidden_nodes])
	Y = tf.placeholder(tf.int64, [None])
	Y_ = tf.one_hot(indices=Y, depth=2) # one_hot labels: [N,M]

	fc1 = tf.layers.dense(inputs=X,units=512,activation=tf.nn.relu)
	fc2 = tf.layers.dense(inputs=fc1,units=512,activation=tf.nn.relu)
	out = tf.layers.dense(inputs=fc2,units=2,activation=None)

	loss = tf.losses.softmax_cross_entropy(logits=out,onehot_labels=Y_)
	accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(out,axis=1),tf.argmax(Y_,axis=1)),dtype=tf.float32))

	optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

	sess = tf.Session()
	sess.run(tf.global_variables_initializer())

	BATCH_SIZE = 128
	for epoch in range(100):
		i = 0
		while i < len(x_train_encoded):
			try:
				batch_x = x_train_encoded[i:i+BATCH_SIZE]
				batch_y = y_train[i:i+BATCH_SIZE]
			except:
				batch_x = x_train_encoded[i:]
				batch_y = y_train[i:]
			i+=BATCH_SIZE
			sess.run(optimizer, feed_dict={X:batch_x, Y:batch_y})

	"""evaluation"""
	avg_testing_acc = sess.run(accuracy, feed_dict={X: x_test_encoded, Y: y_test})
	print("Testing accuracy: ",avg_testing_acc)
	accuracy_sum+=avg_testing_acc

print("==========================================")
print("BP	 ===================================")
print("==========================================")
print("Average time: ", time_sum/10.0)
print("Average accuracy: ", accuracy_sum/10.0)
print("==========================================")

Testing accuracy:  0.26541996
Testing accuracy:  0.26541996
Testing accuracy:  0.26541996
Testing accuracy:  0.26541996
Testing accuracy:  0.73458004
Testing accuracy:  0.73458004
Testing accuracy:  0.26541996
Testing accuracy:  0.26541996
Testing accuracy:  0.73458004
Testing accuracy:  0.73458004
BP	 ===================================
Average time:  60.62157433032989
Average accuracy:  0.45308399200439453


In [6]:
def one_hot(y, classes):
    I = np.eye(classes)
    #print(I)
    ret = []
    l = 0
    for y_ in y:
        ret.append(I[y_])
            
    return np.array(ret)
    
y_train_onehot = one_hot(y_train, 3)
y_test_onehot = one_hot(y_test,3)

In [7]:
print(y_train_onehot)

[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]


In [8]:
border = 10000
x_train_init = x_train[:border]
x_train_seq = x_train[border:]

print('total initial: ', (border))
print('total sequential: ', len(x_train_seq))
print('total testing: ', len(x_test))

total initial:  10000
total sequential:  50000
total testing:  18823


In [9]:
n_input_nodes = 50
n_hidden_nodes = 2  # used to be 6
n_output_nodes = 50

In [10]:
import time
accuracy_sum = 0
time_sum = 0
for PPAP in range(10):
	tf.reset_default_graph()
	# ===========================================
	# Instantiate os-elm
	# ===========================================
	auto_elm = AUTO_ELM(
		# the number of input nodes.
		n_input_nodes=n_input_nodes,
		# the number of hidden nodes.
		n_hidden_nodes=n_hidden_nodes,
		# the number of output nodes.
		n_output_nodes=n_output_nodes,
		# loss function.
		# the default value is 'mean_squared_error'.
		# for the other functions, we support
		# 'mean_absolute_error', 'categorical_crossentropy', and 'binary_crossentropy'.
		c_value = 2.0**(-3),
		loss='mean_squared_error',
		# activation function applied to the hidden nodes.
		# the default value is 'sigmoid'.
		# for the other functions, we support 'linear' and 'tanh'.
		# NOTE: OS-ELM can apply an activation function only to the hidden nodes.
		activation='sin',
	)
	# ===========================================
	# Training
	# ===========================================
	# the initial training phase
	t1 = time.time()
	auto_elm.init_train(x_train_init, x_train_init)
	t2 = time.time()
	time_sum+=(t2-t1)

	# the sequential training phase
	batch_size = 10000

	t1 = time.time()
	for epoch in range(100):
		for i in range(0, len(x_train_seq), batch_size):
			x_batch = x_train_seq[i:i+batch_size]
			if len(x_batch) != batch_size:
				break
			auto_elm.seq_train(x_batch, x_batch)
	t2 = time.time()
	time_sum+=(t2-t1)/100
time_sum = time_sum/10
print(time_sum)

0.15535519623756408


In [11]:
x_train_encoded = auto_elm.encoding(x_train)
x_test_encoded = auto_elm.encoding(x_test)


border = 10000
x_train_init = x_train_encoded[:border]
y_train_init = y_train[:border]
x_train_seq = x_train_encoded[border:]
y_train_seq = y_train[border:]
INPUT_DIMENSION = 2
NUM_CLASSES = 3

print(x_train_encoded.shape)
print(x_test_encoded.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 2)
(18823, 2)
(60000,)
(18823,)


In [12]:
tf.reset_default_graph() # clear all the tensors
sess = tf.Session()


"""Placeholders"""
X_ = tf.placeholder(tf.float32, [None, INPUT_DIMENSION])
#X_ = tf.reshape(X, [-1, INPUT_DIMENSION]) # Flatten X: [N,D]
Y = tf.placeholder(tf.int64, [None]) # labels
Y_ = tf.one_hot(indices=Y, depth=NUM_CLASSES) # one_hot labels: [N,M]

"""Some constants"""
D = INPUT_DIMENSION
M = NUM_CLASSES # Number of outputs
C = tf.constant(2.0**(-1))

"""Weights"""
alpha_1 = tf.get_variable('alpha_1',shape=[D, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False) # 1st subnetwork
alpha_2 = tf.get_variable('alpha_2',shape=[D, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False) # 2st subnetwork
alpha_3 = tf.get_variable('alpha_3',shape=[D, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False)
beta_1 = tf.get_variable('beta_1',shape=[M, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False)
beta_2 = tf.get_variable('beta_2',shape=[M, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False)
beta_3 = tf.get_variable('beta_3',shape=[M, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False)

In [13]:
k = tf.get_variable('k',shape=[D, D],initializer=tf.random_uniform_initializer(-1,1),trainable=False)
m = tf.get_variable('m',shape=[M, M],initializer=tf.random_uniform_initializer(-1,1),trainable=False)

In [14]:
"""Utility functions"""
def mul(A, B):
    return tf.matmul(A, B)

def inv(A):
    return tf.matrix_inverse(A)

def t(A):
    return tf.transpose(A)

def sin(A):
    return tf.math.sin(A)

def asin(A):
    return tf.math.asin(A)

def sqrt(A):
    return tf.sqrt(A)

def sqr(A):
    return tf.math.pow(A, 2)

def pseudo_inv(A, I, C):
    C_I = I/C
    return mul(t(A), inv(C_I + mul(A, t(A))))

def h(A):
    '''activation function'''
    return sin(A)

def h_(A):
    '''inverse activation function'''
    return asin(A)

def u(A):
    '''normalize the input to (0,1]'''
    return tf.math.sigmoid(A) # sigmoid
    
def u_(A):
    '''the inverse of u'''
    ONE = tf.constant(1.0)
    return -(tf.math.log(ONE/A - ONE)) # the inverse of sigmoid
    
def subnet_output(alpha, beta, A):
    return t(mul(beta, h(mul(t(alpha), t(A))))) 

In [15]:
"""Initial Training Graph"""
# D: input dimension
# N: number of input samples
# M: number of classes (number of outputs)
'''some pre-computations'''
X_init = t(X_) # [D,N]
Y_init = t(Y_) # [M,N]
N_init = D # number of dimensions
I_DxD = tf.eye(N_init, dtype=tf.float32) # [D,D]
I_MxM = tf.eye(M, dtype=tf.float32) # [M,M]
C_I = I_DxD/C
H_I = I_MxM/C

add = C_I + mul(X_init, t(X_init))
k = tf.assign(k,add)
X_inv_init = pseudo_inv(X_init, I_DxD, C) # [N,D]

'''1st subnet'''
alpha_1_init_calculated = t(mul(h_(Y_init), X_inv_init)) # ([M,N]x[N,D])T=[D,M]
alpha_1_init = tf.assign(alpha_1, alpha_1_init_calculated) # [D,M]
H_1_init = h(mul(t(alpha_1_init), X_init)) # [M,N]
H_add = H_I + mul(H_1_init,t(H_1_init))
m = tf.assign(m,H_add)
H_pseudo_init = pseudo_inv(H_1_init,I_MxM,C) #[N,M]

beta_1_init_calculated = mul(Y_init, t(H_1_init))/sqr(tf.norm(H_1_init)) # [M,M]
beta_1_init_calculated = mul(Y_init,H_pseudo_init)

beta_1_init = tf.assign(beta_1, beta_1_init_calculated) # [M,M]
H_beta_1_init = mul(beta_1_init, t(mul(t(X_init), alpha_1_init))) # [M,N]
E_1_init = Y_init - H_beta_1_init # [M,N]

'''2nd subnet'''
#alpha_2_init_calculated = t(mul(h_(E_1_init), X_inv_init)) # [D,M]    
alpha_2_init_calculated = t(mul(asin(Y_init), X_inv_init)) # [D,M]

alpha_2_init = tf.assign(alpha_2, alpha_2_init_calculated) # [D,M]
H_2_init = h(mul(t(alpha_2_init), X_init)) # [M,N]
H_2_inv_init = pseudo_inv(H_2_init, I_MxM, C) # [M,N]
H_add = H_I + mul(H_2_init,t(H_2_init))
#m = tf.assign(m,H_add)
H_pseudo_init = pseudo_inv(H_2_init,I_MxM,C) #[N,M]

beta_2_init_calculated = mul(E_1_init, t(H_2_init))/sqr(tf.norm(H_2_init)) # [M,M]
beta_2_init_calculated = mul(E_1_init, H_pseudo_init)

beta_2_init = tf.assign(beta_2, beta_2_init_calculated) # [M,M]
H_beta_2_init = mul(beta_2_init, t(mul(t(X_init), alpha_2_init))) # [M,N]
E_2_init = Y_init - (H_beta_1_init+H_beta_2_init) # [M,N]

'''3rd subnetwork'''
alpha_3_init_calculated = t(mul(h_(E_2_init), X_inv_init)) # [D,M]    
alpha_3_init_calculated = t(mul(asin(Y_init), X_inv_init)) # [D,M]

alpha_3_init = tf.assign(alpha_3, alpha_3_init_calculated) # [D,M]
H_3_init = h(mul(t(alpha_3_init), X_init)) # [M,N]
H_3_inv_init = pseudo_inv(H_3_init, I_MxM, C) # [M,N]

beta_3_init_calculated = mul(E_2_init, t(H_3_init))/sqr(tf.norm(H_3_init)) # [M,M]
beta_3_init_calculated = mul(E_2_init, H_3_inv_init)

beta_3_init = tf.assign(beta_3, beta_3_init_calculated) # [M,M]
H_beta_3_init = mul(beta_3_init, t(mul(t(X_init), alpha_3_init))) # [M,N]
E_3_init = Y_init - (H_beta_3_init+H_beta_2_init+ H_beta_1_init) # [M,N]

#init_train_graph = H_beta_1_init
init_train_graph = E_3_init



In [16]:
"""Evaluate initial-training graph"""
#logits_init =  subnet_output(alpha_1, beta_1, X_)
logits_init = subnet_output(alpha_1, beta_1, X_) + subnet_output(alpha_2, beta_2, X_) + subnet_output(alpha_3, beta_3, X_)
loss_init = tf.losses.mean_squared_error(labels=Y_, predictions=logits_init)
accuracy_init = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y_, axis=1), tf.argmax(logits_init, axis=1)), dtype=tf.float32))

In [17]:
"""Initialize variables"""
init = tf.global_variables_initializer()
sess.run(init)
"""Initial training"""
t1=time.time()
sess.run(E_3_init, feed_dict={X_: x_train_init, Y: y_train_init})
t2 = time.time()
time_sum+=(t2-t1)
print("Initial training done")

"""Initial training evaluation"""
tr_loss, tr_acc = sess.run([loss_init, accuracy_init], feed_dict={X_: x_train_encoded, Y: y_train})
ts_loss, ts_acc = sess.run([loss_init, accuracy_init], feed_dict={X_: x_test_encoded, Y: y_test})
print("Initial train training loss: ", tr_loss)
print("Initial train training accuracy: ", tr_acc)
print("Initial train testing loss: ", ts_loss)
print("Initial train testing accuracy: ", ts_acc)

Initial training done
Initial train training loss:  0.26180092
Initial train training accuracy:  0.49996668
Initial train testing loss:  0.2619312
Initial train testing accuracy:  0.49938905


In [18]:
"""Sequential Training Graph"""
# D: input dimension
# N: number of input samples
# M: number of classes (number of outputs)
X_seq = t(X_) # [D,N]
Y_seq = t(Y_) # [M,N]
pseudo = mul(X_seq, X_) #DXD
k = tf.assign(k, tf.add(k,pseudo)) #DXD
k_inv = inv(k)

new = tf.matmul(tf.matmul(k_inv, X_seq),h_(Y_) - tf.matmul(X_, alpha_1))
alpha1_seq = tf.assign(alpha_1,tf.add(alpha_1,new)) #DXM
H_1_seq = h(mul(t(alpha1_seq), X_seq)) # [M,N]
m_su = mul(H_1_seq,t(H_1_seq))
m = tf.assign(m,tf.add(m,m_su))
m_inv = inv(m)
#update = tf.matmul(tf.matmul(m_inv,H_1_seq),h_(Y_seq)- tf.matmul())
H_pseudo_init = pseudo_inv(H_1_seq,I_MxM,C) #[N,M]
#UPDATE = tf.matmul(tf.matmul(K_inverse, HT), inverse_acti_y - tf.matmul(H, self.__outputWeight))
beta_1_seq_calculated = mul(Y_seq, H_pseudo_init) # [M,M]
beta_1_seq = tf.assign(beta_1, beta_1_seq_calculated) # [M,M]
H_beta_1_seq = mul(beta_1_seq, t(mul(X_, alpha1_seq))) # [M,N]
E_1_seq = Y_seq - H_beta_1_seq # [M,N]

'''2nd subnetwork'''

new = tf.matmul(tf.matmul(k_inv, X_seq),h_(Y_) - tf.matmul(X_, alpha_2))
alpha2_seq = tf.assign(alpha_2,tf.add(alpha_2,new)) #DXM
H_2_seq = h(mul(t(alpha2_seq), X_seq)) # [M,N]
H_pseudo_init = pseudo_inv(H_2_seq,I_MxM,C) #[N,M]
beta_2_seq_calculated = mul(E_1_seq, H_pseudo_init) # [M,M]
beta_2_seq = tf.assign(beta_2, beta_2_seq_calculated) # [M,M]
H_beta_2_seq = mul(beta_2_seq, t(mul(t(X_seq), alpha2_seq))) # [M,N]
E_2_seq = Y_seq - (H_beta_2_seq+ H_beta_1_seq) # [M,N]

'''3rd subnetwork'''
new = tf.matmul(tf.matmul(k_inv, X_seq),h_(Y_) - tf.matmul(X_, alpha_3))
alpha3_seq = tf.assign(alpha_3,tf.add(alpha_3,new)) #DXM
H_3_seq = h(mul(t(alpha3_seq), X_seq)) # [M,N]
H_pseudo_init = pseudo_inv(H_3_seq,I_MxM,C) #[N,M]
beta_3_seq_calculated = mul(E_2_seq, H_pseudo_init) # [M,M]
beta_3_seq = tf.assign(beta_3, beta_3_seq_calculated) # [M,M]
H_beta_3_seq = mul(beta_3_seq, t(mul(t(X_seq), alpha3_seq))) # [M,N]
E_3_seq = Y_seq - (H_beta_3_seq +H_beta_2_seq + H_beta_1_seq )# [M,N]
seq_train_graph = E_3_seq




In [19]:
"""Evaluate initial-training graph"""
logits_seq =  subnet_output(alpha_1, beta_1, X_) + subnet_output(alpha_2, beta_2, X_) + subnet_output(alpha_3, beta_3, X_)
#logits_init = subnet_output(alpha_1, beta_1, X_) + subnet_output(alpha_2, beta_2, X_)
loss_seq = tf.losses.mean_squared_error(labels=Y_, predictions=logits_seq)
accuracy_seq = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y_, axis=1), tf.argmax(logits_seq, axis=1)), dtype=tf.float32))

In [20]:
"""Sequential training"""
batch_size = 10000

epoch_train_accuracy = []
epoch_test_accuracy = []
for epoch in range(10):
    #pbar = tqdm.tqdm(total=len(x_train), desc='sequential training phase')
    t1 = time.time()
    for i in range(0, len(x_train_seq), batch_size):
        x_batch = x_train_seq[i:i+batch_size]
        y_batch = y_train_seq[i:i+batch_size]
        if len(x_batch) != batch_size:
            break
        sess.run(seq_train_graph, feed_dict={X_: x_batch, Y: y_batch})
        #pbar.update(n=len(x_batch))
    t2 = time.time()
    time_sum+=(t2-t1)
    '''epoch evaluation'''
    [train_loss, train_accuracy] = sess.run([loss_seq, accuracy_seq], feed_dict={X_: x_train_encoded, Y: y_train})
    [test_loss, test_accuracy] = sess.run([loss_seq, accuracy_seq], feed_dict={X_: x_test_encoded, Y: y_test})
    print('train_loss: %f, train_accuracy: %f' % (train_loss, train_accuracy))
    print('test_loss: %f, test_accuracy: %f' % (test_loss, test_accuracy))
    epoch_train_accuracy.append(train_accuracy)
    epoch_test_accuracy.append(test_accuracy)
#sess.run(init_train_graph, feed_dict={X: x_train_init, Y: y_train_init})
print("Sequential training done")

"""Sequential training evaluation"""
#tr_loss, tr_acc = sess.run([loss_init, accuracy_init], feed_dict={X: x_train, Y: y_train})
#ts_loss, ts_acc = sess.run([loss_init, accuracy_init], feed_dict={X: x_test, Y: y_test})
#print("Sequential train training loss: ", tr_loss)
#print("Sequential train training accuracy: ", tr_acc)
#print("Sequential train testing loss: ", ts_loss)
#print("Sequential train testing accuracy: ", ts_acc)

train_loss: 0.268546, train_accuracy: 0.504083
test_loss: 0.268873, test_accuracy: 0.503745
train_loss: 0.268522, train_accuracy: 0.504083
test_loss: 0.268850, test_accuracy: 0.503745
train_loss: 0.268548, train_accuracy: 0.504083
test_loss: 0.268876, test_accuracy: 0.503745
train_loss: 0.268533, train_accuracy: 0.504083
test_loss: 0.268862, test_accuracy: 0.503745
train_loss: 0.268533, train_accuracy: 0.504083
test_loss: 0.268862, test_accuracy: 0.503745
train_loss: 0.268533, train_accuracy: 0.504083
test_loss: 0.268862, test_accuracy: 0.503745
train_loss: 0.268533, train_accuracy: 0.504083
test_loss: 0.268862, test_accuracy: 0.503745
train_loss: 0.268533, train_accuracy: 0.504083
test_loss: 0.268862, test_accuracy: 0.503745
train_loss: 0.268533, train_accuracy: 0.504083
test_loss: 0.268862, test_accuracy: 0.503745
train_loss: 0.268533, train_accuracy: 0.504083
test_loss: 0.268862, test_accuracy: 0.503745
Sequential training done


'Sequential training evaluation'

In [23]:
time_sum/10

0.03723330309391022

In [21]:
from ELM import ELM 
tf.reset_default_graph()
# ===========================================
# Instantiate os-elm
# ===========================================

    
def one_hot(y, classes):
    I = np.eye(classes)
    ret = []
    for y_ in y:
        ret.append(I[y_])
    return np.array(ret)
    
#y_train_onehot = one_hot(y_train, 6)
#y_test_onehot = one_hot(y_test, 6)
    
model = ELM(n_input_nodes=10,
                    n_hidden_nodes=10,
                    n_output_nodes=3,
                    loss='mean_squared_error',
                    activation='sigmoid',
                    name='elm',
                    )
   
feature_train = auto_elm.encoding(x_train)
feature_test = auto_elm.encoding(x_test)

model.fit(feature_train, y_train_onehot)

test_loss, test_acc = model.evaluate(feature_test, y_test_onehot, metrics=['loss', 'accuracy'])
print('test_loss: %f' % test_loss)
print('test_acc: %f' % test_acc)
#accuracy_sum+=test_acc
   



    
print("==========================================")
print("OS-ELA ===================================")
print("==========================================")
#print("Average time: ", time_sum/10.0)
#print("Average accuracy: ", accuracy_sum/10.0)
print("==========================================")




        
            
        

ValueError: shapes (60000,2) and (10,10) not aligned: 2 (dim 1) != 10 (dim 0)

In [ ]:
pred = np.argmax(model.predict(feature_test),axis=1)
test_accuracy = np.sum(np.equal(pred, y_test_onehot))/len(pred)
print(test_accuracy)

In [ ]:
print(y_test[:,0])

In [ ]:
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.87)
pca.fit(x_train)
train_img = pca.transform(x_train)
test_img  = pca.transform(x_test)
train_img.shape,test_img.shape

In [ ]:
from ELM import ELM 
tf.reset_default_graph()
# ===========================================
# Instantiate os-elm
# ===========================================

    
def one_hot(y, classes):
    I = np.eye(classes)
    ret = []
    for y_ in y:
        ret.append(I[y_])
    return np.array(ret)
    
#y_train_onehot = one_hot(y_train, 6)
#y_test_onehot = one_hot(y_test, 6)
    
model = ELM(n_input_nodes=10,
                    n_hidden_nodes=10,
                    n_output_nodes=3,
                    loss='mean_squared_error',
                    activation='sigmoid',
                    name='elm',
                    )
   
feature_train = train_img
feature_test = test_img

model.fit(feature_train, y_train_onehot)

test_loss, test_acc = model.evaluate(feature_test, y_test_onehot, metrics=['loss', 'accuracy'])
print('test_loss: %f' % test_loss)
print('test_acc: %f' % test_acc)
#accuracy_sum+=test_acc
   



    
print("==========================================")
print("OS-ELA ===================================")
print("==========================================")
#print("Average time: ", time_sum/10.0)
#print("Average accuracy: ", accuracy_sum/10.0)
print("==========================================")




        
            
        

In [ ]:
border = 30000
x_train_init = feature_train[:border]
y_train_init = y_train_onehot[:border]
x_train_seq = feature_train[border:]
y_train_seq = y_train_onehot[border:]

In [ ]:
tf.reset_default_graph()
from os_MFB1 import MFB_ELM 

In [ ]:
C = 4
mfb_elm = MFB_ELM(
    # the number of input nodes.
    n_input_nodes=n_input_nodes,
    # the number of hidden nodes.
    n_hidden_nodes=n_hidden_nodes,
    # the number of output nodes.
    n_output_nodes=n_output_nodes,
    #C = C,
    # loss function.
    # the default value is 'mean_squared_error'.
    # for the other functions, we support
    # 'mean_absolute_error', 'categorical_crossentropy', and 'binary_crossentropy'.
    loss='mean_squared_error',
    # activation function applied to the hidden nodes.
    # the default value is 'sigmoid'.
    # for the other functions, we support 'linear' and 'tanh'.
    # NOTE: OS-ELM can apply an activation function only to the hidden nodes.
    activation='sin',
    C = C
)

In [ ]:
mfb_elm.init_train(x_train_init, y_train_init)

'''initial training evaluation'''
[train_loss, train_accuracy] = mfb_elm.evaluate(feature_train[:len(feature_test)], y_train_onehot[:len(feature_test)], metrics=['loss', 'accuracy'])
[test_loss, test_accuracy] = mfb_elm.evaluate(feature_test, y_test_onehot, metrics=['loss', 'accuracy'])
print('train_loss: %f, train_accuracy: %f' % (train_loss, train_accuracy))
print('test_loss: %f, test_accuracy: %f' % (test_loss, test_accuracy))
    
epoch_train_accuracy = []
epoch_test_accuracy = []

In [ ]:
batch_size = 1000


for epoch in range(10):
    #pbar = tqdm.tqdm(total=len(x_train), desc='sequential training phase')
    for i in range(0, len(x_train_seq), batch_size):
        x_batch = x_train_seq[i:i+batch_size]
        y_batch = y_train_seq[i:i+batch_size]
        if len(x_batch) != batch_size:
            break
        mfb_elm.seq_train(x_batch, y_batch)
       # pbar.update(n=len(x_batch))
    '''epoch evaluation'''
    [train_loss, train_accuracy] = mfb_elm.evaluate(feature_train, y_train_onehot, metrics=['loss', 'accuracy'])
    [test_loss, test_accuracy] = mfb_elm.evaluate(feature_test, y_test_onehot, metrics=['loss', 'accuracy'])
    print('train_loss: %f, train_accuracy: %f' % (train_loss, train_accuracy))
    print('test_loss: %f, test_accuracy: %f' % (test_loss, test_accuracy))
    epoch_train_accuracy.append(train_accuracy)
    epoch_test_accuracy.append(test_accuracy)

In [ ]:
np.random.seed(1337)  # for reproducibility
#from sklearn.datasets import load_digits
#from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import accuracy_score

from dbn.tensorflow import SupervisedDBNClassification

classifier = SupervisedDBNClassification(hidden_layers_structure=[256, 256],
                                         learning_rate_rbm=0.05,
                                         learning_rate=0.01,
                                         n_epochs_rbm=10,
                                         n_iter_backprop=100,
                                         batch_size=32,
                                         activation_function='relu',
                                         dropout_p=0.2)
classifier.fit(x_train, y_train)
Y_pred = classifier.predict(x_test)
print('Done.\nAccuracy: %f' % accuracy_score(y_test, Y_pred))